### Предобработка текста

In [8]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("https://storage.yandexcloud.net/auth-def-2024/datasets/meta_table_with_texts.csv")
df = df[['author', 'text']]
df.head(5)

,author,text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...


Загружаем лемматизированные тексты из заранее подготовленного файла, сформированного после экспериментов.

In [4]:
df_lemm = pd.read_csv('lemm_texts.csv')
df_lemm.head()

,author,lemm_text
0,Пушкин Александр Сергеевич,литературный альбомъ сразить рыцарь послднимъ ...
1,Карамзин Николай Михайлович,достоинство древний новый перевод немецкий нек...
2,Гоголь Николай Васильевич,полный собрание сочинение письмо так переписка...
3,Мамин-Сибиряк Дмитрий Наркисович,собрана сочиненйтомъ восьмой издан марксъ петр...
4,Мамин-Сибиряк Дмитрий Наркисович,нимфа щегольский волжский пароход вулкан дать ...


In [5]:
df['lemm_text'] = df_lemm['lemm_text']
df.head()

,author,text,lemm_text
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ сразить рыцарь послднимъ ...
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,достоинство древний новый перевод немецкий нек...
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,полный собрание сочинение письмо так переписка...
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,собрана сочиненйтомъ восьмой издан марксъ петр...
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,нимфа щегольский волжский пароход вулкан дать ...


In [7]:
# Удаление невалидной записи с английским текстом
df = df.drop(index=1918)
df = df.reset_index(drop=True)
len(df)

2564

In [9]:
# Токенизация текста
def words_list(x):
  # Подсчёт слов в тексте
  words = re.findall(r'\b\w+\b', x)
  return [w.lower() for w in words]

In [10]:
df['words'] = df['lemm_text'].apply(words_list)
df.head(5)

,author,text,lemm_text,words
0,Пушкин Александр Сергеевич,"\n \nЛитературный альбомъ.\n""Сраженный рыцар...",литературный альбомъ сразить рыцарь послднимъ ...,"[литературный, альбомъ, сразить, рыцарь, послд..."
1,Карамзин Николай Михайлович,\nО достоинстве древних и новых\n(Перевод с не...,достоинство древний новый перевод немецкий нек...,"[достоинство, древний, новый, перевод, немецки..."
2,Гоголь Николай Васильевич,\n Гоголь Н. В. Полное собрание сочинений и ...,полный собрание сочинение письмо так переписка...,"[полный, собрание, сочинение, письмо, так, пер..."
3,Мамин-Сибиряк Дмитрий Наркисович,\n \nД. МАМИНЪ-СИБИРЯКЪПОЛНОЕ СОБРАНІЕ СОЧИН...,собрана сочиненйтомъ восьмой издан марксъ петр...,"[собрана, сочиненйтомъ, восьмой, издан, марксъ..."
4,Мамин-Сибиряк Дмитрий Наркисович,\nДмитрий Мамин-Сибиряк\nНимфа\nI.\n Щегольс...,нимфа щегольский волжский пароход вулкан дать ...,"[нимфа, щегольский, волжский, пароход, вулкан,..."


Загрузим эвристики, которые были рассчитаны и отобраны на этапе проведения экспериментов и замера метрик качества моделей.

In [11]:
# Загрузка фрейма с эвристиками
df_heuristics = pd.read_csv('heuristics.csv')
df_heuristics.head(5)

,num_words,avg_sentence_length,avg_word_length
0,347,17.350000,5.002882
1,988,17.642857,5.452429
2,177463,13.357143,5.226318
3,8774,12.410184,5.284477
4,4221,12.234783,5.027008


### Формирование выборок

На этапе проведения экспериментов было принято решение использовать комбинированный подход в выборе признаков.

In [14]:
X_heuristics = df_heuristics.copy()
X_words = df['words'].copy()
y = df['author'].copy()

In [19]:
from sklearn.model_selection import train_test_split

# Обучающая и тренировочная выборки
X_heur_train, X_heur_test, X_words_train, X_words_test, y_train, y_test = train_test_split(X_heuristics, X_words, y, test_size=0.2, random_state=42)

Обучение модели для дальнейшего формирования эмбеддингов на тренировочных данных:

In [20]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=X_words_train, vector_size=300, window=5, min_count=1, workers=4)

In [21]:
import numpy as np

# Функция для представления текста как среднего из векторов слов
def vectorize_text(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)  # Если нет слов из текста, возвращаем нулевой вектор
    return np.mean(vectors, axis=0)

In [22]:
X_train_vect = np.array([vectorize_text(text, w2v_model) for text in X_words_train])
X_test_vect = np.array([vectorize_text(text, w2v_model) for text in X_words_test])

In [32]:
# Формирование итоговых датасетов
X_train_combined = np.hstack([X_heur_train, X_train_vect])
X_test_combined = np.hstack([X_heur_test, X_test_vect])

### Создание пайплайна

На этапе проведения экспериментов было выявлено, что лучшие метрики качества выдавала логистическая регрессия. Поэтому в пайплайне указываем `LogisticRegression`:

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Пайплайн
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline.fit(X_train_combined, y_train)
y_pred = pipeline.predict(X_test_combined)

In [37]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

# Оценка качества модели
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Precision, Recall, F1 (взвешенные для многоклассовой задачи)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.8089668615984406
Precision: 0.8092140200780344
Recall: 0.8089668615984406
F1 Score: 0.8081065574323371
Confusion Matrix:
 [[13  2  0  1  1  0  3  0  0  0  1  0  1]
 [ 1 42  0  2  0  0  1  0  0  0  1  0  2]
 [ 0  0 30  2  0  1  0  2  0  2  1  1  1]
 [ 0  0  1 34  0  0  2  0  0  1  3  3  0]
 [ 0  1  1  0  8  0  0  0  0  1  0  0  1]
 [ 0  0  1  0  0 51  0  0  0  1  0  2  1]
 [ 1  2  2  1  0  0 57  0  0  1  0  0  2]
 [ 0  0  1  0  0  1  1  4  0  0  0  2  0]
 [ 0  0  0  0  0  0  2  0 50  0  0  1  0]
 [ 1  0  0  0  1  2  2  2  0 24  0  2  0]
 [ 1  1  1  1  0  0  0  0  0  0 30  3  0]
 [ 0  0  1  3  0  0  2  0  0  3  0 56  0]
 [ 1  1  1  4  1  0  1  0  0  0  0  1 16]]


По итогу на комбинированном наборе фичей (эвристики + текстовые) логистическая регрессия показала значения метрик точности предсказания в районе $\approx 81\%$.

Сохранение моделей в `.pkl` файлы:

In [ ]:
!pip install joblib

In [ ]:
from joblib import dump, load

# Сохранение моделей в файл
dump(w2v_model, 'w2v_model.pkl')
dump(pipeline, 'pipeline.pkl')  # Сохранение Logistic Regression